In [2]:
import pandas as pd
import numpy as np
from IPython.display import display

## Removing NA Values

In [3]:
df = pd.DataFrame({
    "first": ["Matt", "Eli", "Tom"],
    "initial": ["H", np.nan, "S"],
    "last": ["Mazur", "Smith", np.nan]
})

print("Original:\n\n", df)
print("\nRemoving all rows with NA:\n\n", df.dropna())
print("\nRemoving just last names with NA:\n\n", df.dropna(subset=["last"]))
print("\nRemoving all columns with NA:\n\n", df.dropna(axis=1))

Original:

   first initial   last
0  Matt       H  Mazur
1   Eli     NaN  Smith
2   Tom       S    NaN

Removing all rows with NA:

   first initial   last
0  Matt       H  Mazur

Removing just last names with NA:

   first initial   last
0  Matt       H  Mazur
1   Eli     NaN  Smith

Removing all columns with NA:

   first
0  Matt
1   Eli
2   Tom


## Lagged Values

In [5]:
df = pd.DataFrame({
    "price": range(1, 11)
})
df["last_price"] = df["price"].shift()
df

,price,last_price
0,1,NaN
1,2,1.0
2,3,2.0
3,4,3.0
4,5,4.0
5,6,5.0
6,7,6.0
7,8,7.0
8,9,8.0


## Lagged Values in a Group with Trailing Averages

In [18]:
df = pd.DataFrame({
    "stock": ["AAPL", "AAPL", "AAPL", "GOOG", "GOOG", "GOOG", "GOOG", "T", "T", "T"],
    "price": range(1, 11)
})

df["last_price"] = df.groupby("stock")["price"].shift()

# This row and the row prior to it
df["mean_price_including_current"] = df.groupby("stock")["price"].rolling(2).mean().reset_index(0, drop=True)

# The two prior rows (and not this one)
df["mean_price_not_including_current"] = df.groupby("stock")["last_price"].rolling(2).mean().reset_index(0, drop=True)

# If there are two prior values, it will average them, but 1 is fine
df["mean_price_not_including_current_no_min"] = df.groupby("stock")["last_price"].rolling(2, min_periods=1).mean().reset_index(0, drop=True)

df

,stock,price,last_price,mean_price_including_current,mean_price_not_including_current,mean_price_not_including_current_no_min
0,AAPL,1,NaN,NaN,NaN,NaN
1,AAPL,2,1.0,1.5,NaN,1.0
2,AAPL,3,2.0,2.5,1.5,1.5
3,GOOG,4,NaN,NaN,NaN,NaN
4,GOOG,5,4.0,4.5,NaN,4.0
5,GOOG,6,5.0,5.5,4.5,4.5
6,GOOG,7,6.0,6.5,5.5,5.5
7,T,8,NaN,NaN,NaN,NaN
8,T,9,8.0,8.5,NaN,8.0
9,T,10,9.0,9.5,8.5,8.5


## Subsetting

In [60]:
df = pd.DataFrame({
    "index": range(1, 11),
    "price": np.random.randint(0, 100, size=10),
    "other": np.random.randint(1000, 2000, size=10)
}).set_index("index")

display(df)

select_rows_by_column_value = df[df["price"] > 50]
display(select_rows_by_column_value)

select_rows_by_index_value = df.loc[[3, 4, 5, 6, 7]]
display(select_rows_by_index_value)

select_rows_by_index_value = df[df.index.isin([3, 4, 5, 6, 7])]
display(select_rows_by_index_value)

select_rows_and_columns = df.loc[df["price"] > 50, ["price"]]
display(select_rows_and_columns)


,price,other
index,,
1,43,1029
2,49,1015
3,38,1303
4,9,1039
5,8,1014
6,9,1397
7,84,1138
8,98,1120
9,83,1121


,price,other
index,,
7,84,1138
8,98,1120
9,83,1121


,price,other
index,,
3,38,1303
4,9,1039
5,8,1014
6,9,1397
7,84,1138


,price,other
index,,
3,38,1303
4,9,1039
5,8,1014
6,9,1397
7,84,1138


,price
index,
7,84
8,98
9,83


## Updating values of a subset

In [10]:
df = pd.DataFrame({
    "stock": ["AAPL", "GOOG"],
    "price": [5, 40]
})

display(df)

# Double the price of values that meet a certain criteria
df.loc[df["stock"] == "GOOG", "price"] = df.loc[df["stock"] == "GOOG", "price"] * 2

display(df)

,stock,price
0,AAPL,5
1,GOOG,40


,stock,price
0,AAPL,5
1,GOOG,80


## Filling NA values

In [3]:
df = pd.DataFrame({
    "index": range(1, 6),
    "price": [20, 40, np.nan, 60, np.nan]
}).set_index("index")

display(df)

# Avoid using inplace
df.loc[3] = df.loc[3].fillna(0)

display(df)

,price
index,
1,20.0
2,40.0
3,NaN
4,60.0
5,NaN


,price
index,
1,20.0
2,40.0
3,0.0
4,60.0
5,NaN


## Summing Columns Containing NAs

In [14]:
df = pd.DataFrame({
    "a": [np.nan, 4, 5],
    "b": [10, np.nan, 6]
})

display(df)

df["sum"] = df["a"].fillna(0) + df["b"].fillna(0)

display(df)

,a,b
0,NaN,10.0
1,4.0,NaN
2,5.0,6.0


,a,b,sum
0,NaN,10.0,10.0
1,4.0,NaN,4.0
2,5.0,6.0,11.0


## Joining with shared indeces

In [18]:
df1 = pd.DataFrame({
    "company": ["AAPL", "GOOG"],
    "price": [10, 20]
}).set_index("company")

df2 = pd.DataFrame({
    "company": ["AAPL", "GOOG"],
    "ceo": ["Tim Cook", "Sundar Pichai"]
}).set_index("company")

display(df1)
display(df2)

df3 = df1.join(df2)
display(df3)

,price
company,
AAPL,10
GOOG,20


,ceo
company,
AAPL,Tim Cook
GOOG,Sundar Pichai


,price,ceo
company,,
AAPL,10,Tim Cook
GOOG,20,Sundar Pichai


## Joining without shared indeces

In [25]:
df1 = pd.DataFrame({
    "company": ["AAPL", "GOOG", "TSLA"],
    "price": [10, 20, 30]
})

df2 = pd.DataFrame({
    "company": ["AAPL", "GOOG"],
    "ceo": ["Tim Cook", "Sundar Pichai"]
})

# Use `merge` instead of `join`
df3 = pd.merge(df1, df2, on="company")
display(df3)

,company,price,ceo
0,AAPL,10,Tim Cook
1,GOOG,20,Sundar Pichai


## Viewing column names

In [31]:
df = pd.DataFrame({
    "company": ["AAPL", "GOOG", "TSLA"],
    "price": [10, 20, 30]
})

# to_list is an alias of tolist
print("Columns:", df.columns.tolist())
print("Columns:", df.columns.to_list())

Columns: ['company', 'price']
Columns: ['company', 'price']


## Grouping

In [40]:
df = pd.DataFrame({
    "stock": ["AAPL", "AAPL", "AAPL", "GOOG", "GOOG", "GOOG", "GOOG", "T", "T", "T"],
    "price": range(1, 11)
})

display(df)

with_index = df.groupby("stock")["price"].mean()
print("With Index:\n\n", with_index)

without_index = df.groupby("stock", as_index=False)["price"].mean()
print("\nWithout Index:\n\n", without_index)


,stock,price
0,AAPL,1
1,AAPL,2
2,AAPL,3
3,GOOG,4
4,GOOG,5
5,GOOG,6
6,GOOG,7
7,T,8
8,T,9
9,T,10


With Index:

 stock
AAPL    2.0
GOOG    5.5
T       9.0
Name: price, dtype: float64

Without Index:

   stock  price
0  AAPL    2.0
1  GOOG    5.5
2     T    9.0
